In [1]:
import ee
import geemap
import geopandas as gpd
import json
import os
import sys
from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import fiona
#sys.path.insert(1, 'D:/!!Research/rgee_test/GEE_LPC/python_scripts')
sys.path.insert(1, 'C:/Users/Justin-Laptop/Documents/Research/GEE_LPC/python_scripts')
from my_gee_functions import file_address_func
from my_gee_functions import ee_list_func
from my_gee_functions import stdrd_func

In [2]:
# Authenticate if needed
ee.Authenticate()

True

In [2]:
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

In [3]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
shp_file_path = 'C:/Users/Justin-Laptop/Documents/Research/inat_shrubs/southwest_shrubs_shp/'
southwest_shrubs_shp = gpd.read_file(f"{shp_file_path}southwest_shrubs.shp")
#print(southwest_shrubs_shp.crs)
southwest_shrubs_wgs84 = southwest_shrubs_shp.to_crs('epsg:4326')
southwest_shrubs_wgs84.to_file(f'{shp_file_path}sw_shrubs_wgs84.geojson', driver = 'GeoJSON')


In [8]:
southwest_shrubs = geemap.geojson_to_ee(f'{shp_file_path}sw_shrubs_wgs84.geojson')

# Export an ee.FeatureCollection as an Earth Engine asset.
#sw_shrub_export_task = ee.batch.Export.table.toAsset(
#    collection = southwest_shrubs,
#    description = 'sw_shrubs_fc',
#    assetId='projects/ee-jdawsey/assets/sw_shrubs_fc',
#)
#sw_shrub_export_task.start()

EEException: Request payload size exceeds the limit: 10485760 bytes.

### Adding buffers to each of the points in the collection

In [8]:
# Load an example feature collection (this is a sample, replace with your feature collection)
points = southwest_shrubs

# Define a buffer distance in meters
buffer_distance = 100000  # Example: 1000 meters

# Function to buffer each feature
def buffer_feature(feature):
    # Apply the buffer and retain the properties
    buffered_geometry = feature.geometry().buffer(buffer_distance)
    return ee.Feature(buffered_geometry).copyProperties(feature)

# Apply the buffer function to each feature in the collection
buffered_points = points.map(buffer_feature)



In [9]:
# Get the number of features in the grid
num_buffers = buffered_points.size()

# Initialize a counter for unique numbers
unique_number = 1

# Create an empty feature collection to store the modified cells
points_buffers_fc = ee.FeatureCollection([])

# Iterate over each cell in the grid and add a unique number
for i in range(num_buffers.getInfo()):
    buffer = ee.Feature(buffered_points.toList(num_buffers).get(i))
    buffer_with_number = buffer.set('number', unique_number)
    points_buffers_fc = points_buffers_fc.merge(buffer_with_number)
    unique_number += 1

# Save the modified grid as a feature collection in your Earth Engine account
#ee.data.createAsset({'type': 'FeatureCollection'}, 'projects/ee-jdawsey/assets/states_shrubs', tx_modified_grid)

# Export an ee.FeatureCollection as an Earth Engine asset.
buffer_export_task = ee.batch.Export.table.toAsset(
    collection = points_buffers_fc,
    description = 'points_buffers_fc',
    assetId='projects/ee-jdawsey/assets/sw_points_buffers_fc',
)
buffer_export_task.start()

EEException: Request payload size exceeds the limit: 10485760 bytes.

### Importing the new buffers feature class
For each of the buffers in the feature class, need find only the points that overlap with a given buffer and add environmental data to it.

Finding the points that overlap

In [ ]:
# Function to find points that intersect with buffered areas
def find_intersecting_points(buffer):
    intersecting_points = target_points.filterBounds(buffer.geometry())
    return intersecting_points

for buffer in buffers...:
    # Apply the function to find intersecting points for each buffer
    intersecting_collections = buffered_points.map(lambda buffer: ee.FeatureCollection(find_intersecting_points(buffer)))

    # Flatten the list of feature collections into a single feature collection
    flattened_intersecting_points = intersecting_collections.flatten()